In [ ]:
from flightsql import connect, FlightSQLClient

import ibis
from ibis import _

import altair as alt

alt.data_transformers.enable("vegafusion")

from backend import FlightSQLDataFusionBackend

In [ ]:
ibis.options.verbose = True
ibis.options.sql.default_limit = 100

conn = FlightSQLDataFusionBackend.from_connection(
    connect(FlightSQLClient(host="localhost", port=50051, insecure=True))
)

In [ ]:
day_aggs = conn.table("day_aggs_v2", database="external.us_stocks_all")

df = (
    day_aggs.order_by(_.window_start.asc())
    .filter(_.ticker == "AAPL")
    .mutate(log_close=_.close.log())
    .mutate(log_close_diff=_.log_close.lead() - _.log_close)
)

df.execute()

In [ ]:
alt.layer(
    alt.Chart(df).mark_line(color="blue").encode(x="window_start", y="log_close_diff")
).interactive()